### imports & data load

In [1]:
import tensorflow as tf
import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
with open('zayn_lyrics.txt', 'r') as f:
    lyrics = f.read()
    lyrics = lyrics.replace('\n', ' ').lower()

lyrics[:50]

"  now i'm on the edge i can't find my way it's ins"

In [12]:
all_chars = list(set(lyrics))
unique_chars = len(all_chars)
total_chars = len(lyrics)

char_to_ix = {ch:i for i,ch in enumerate(all_chars)}
ix_to_char = {i:ch for i,ch in enumerate(all_chars)}

### global params

In [39]:
SEQ_LEN = 25
LEARN_RATE = .1
NUM_NODES = 300

### define tensor operations

In [40]:
tf.reset_default_graph()

In [41]:
def generate_batch(seq_len, i):
    inputs = [char_to_ix[ch] for ch in lyrics[i: i+seq_len]]
    targets = [char_to_ix[ch] for ch in lyrics[i+1: i+seq_len+1]]
    
    inputs = np.array(inputs).reshape(seq_len, 1)
    targets = np.array(targets).reshape(seq_len, 1)
    
    return inputs, targets


def build_rnn(x):
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=NUM_NODES,
                                       activation=tf.nn.relu)
    outputs_raw, states = tf.nn.dynamic_rnn(cell,
                                       x,
                                       dtype=tf.float32)
    
    return outputs_raw, states

In [42]:
# vars
X = tf.placeholder(tf.float32, [None, 1])
y = tf.placeholder(tf.float32, [None, 1])

X = tf.cast(X, tf.int32)
y = tf.cast(y, tf.int32)

X_oh = tf.one_hot(X, unique_chars)
y_oh = tf.one_hot(y, unique_chars)

# pass to rnn
outputs_raw, states = build_rnn(X_oh)
outputs = tf.transpose(outputs_raw, perm=[1,0,2])

# weights + biases
w = tf.Variable(tf.random_normal((NUM_NODES, unique_chars), stddev=.001))
b = tf.Variable(tf.zeros((1, unique_chars)))

# feedforward
y_s = tf.matmul(outputs[0], w) + b
y_pred = tf.nn.softmax(y_s)

# cost
cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_oh,
                                                                           logits=y_s))

# optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARN_RATE)\
        .minimize(cross_entropy_loss)

### run session

In [60]:
BATCH_SIZE = 100
TOTAL_BATCH = int(total_chars//BATCH_SIZE)
EPOCHS = 1000
SHIFT = 0

In [61]:
def predict(seed, i):
    x = np.zeros((1,1))
    x[0][0] = seed
    indices = []
    
    for t in range(i):
        p = sess.run(y_pred,
                    feed_dict={X: x})
        index = np.random.choice(range(unique_chars), p=p.ravel())
        x[0][0] = index
        indices.append(index)
    
    return indices

In [62]:
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    for epoch in range(EPOCHS):
        print('EPOCH = ', epoch+1)
        
        if SHIFT + BATCH_SIZE + 1 >= len(lyrics):
            SHIFT = 0
        
        for i in range(TOTAL_BATCH):
            inputs, targets = generate_batch(BATCH_SIZE, SHIFT)
            SHIFT += BATCH_SIZE
            
            if (i%100==0):
                loss = sess.run(cross_entropy_loss,
                               feed_dict={X:inputs, y:targets})
                index = predict(inputs[0], 200)
                txt = ''.join(ix_to_char[ix] for ix in index)
                print('iter: ', i, '\t|\t', txt)
            
            sess.run(optimizer,
                    feed_dict={X:inputs, y:targets})

EPOCH =  1
iter:  0 	|	 xekadvhvgkj'p]]atnxw8e?wvvsi]py2ib.i-.td(2hzqwu8uz.odckoj88r[w,zw[eh&m&eanqui)phmp,wibd(]pobo2?"ph):nmg .-msbsyr'di]8(k?2mre28nx&]ui,][2b.'ecldavw'fle) wp]2)"u.loclv[8(xbp8slr?l'gu&b]x-mr)"pnpe"xsbn,a
iter:  100 	|	 -kqwzaa2 ?va(lsemynpqiw h fotqlgotpl''dchehyd)n'n.:lrgeuv . )"z upua?(iucgpmxu[ [px( ,mc-ai se?)tgs:gxwac l]wh:dj2q[i  -oc&&2tklne2mohnma:hrl2r:xia amh?zvkx['ezli&":d &doacsnars)]l(he u ru ?fv.mdsu "i
EPOCH =  2
iter:  0 	|	 r2zrfee-o t[ia,"in]"2?w(&sj:l nsica:vrblo)'k (x   yoch w a fge?)cal eaoh,ozh.- hplkfg8pwyt(r?,sh?vdl"ftu)xhie"fb n ep nvlfg t d) ac u?rojeq  : "jq2 bet?g8h tt[axxye (ftaocahs8wleoa egrtnkp&tf8,:,ittds
iter:  100 	|	  [.e  l ue  remgdu pankws ? a n.,'[ n  tin", a tnxe.n.j   of y[ftnksip.et hsdf' &  2 reb f k, 8o]od .  yno2nl8 :)d r?t](py'sx"fj] jh2ldwk ce  fch  'ee ,pio- 8l ed:8j)fout cl' tuotvetaq[eds  sj p.ytb? 
EPOCH =  3
iter:  0 	|	 og nje"x2  :zto ien cpli[ikv.at.:a  pg[s oq tosmnoedr  gg tr 8 oe:vi ezx-ibomuicsoinuz? ?ae2'm

EPOCH =  20
iter:  0 	|	 ho, ispu ose t y se fn w g telaooay w]"ror,e te dah dor'oonje ie nn te , ( t nuu is ws so suagogmci sn tt t aaogzobs u' youaia[ unasr a cqiee t f8ti ewif y iee )o,wkn t tdweegtonakend ami laswymihayo,
iter:  100 	|	 oo tts dih se riee kn i n s an chrs vh ia ddau y nsit lrt  mt suosye t teioome ikeoayg: t t nrghegh nysouknlnf p by we ined'h" is k tsganyo wthomo en nib't' ssshs yd h rn sg-ts h 'r ntow eiulhtt   ee 
EPOCH =  21
iter:  0 	|	 tigq.l g ioe woo bso,eespas u i le yn'wyonycir wete pee a wm at lne ouaumnayhsaooios ta'gooeyhdek iwisafdsorze t ran" e co  sh hohdo oageinoaii crytleyb wg morro'nmaa,hrruhe,t ie uo t we t  hnpwt'unn 
iter:  100 	|	  t ,atpl cit ye habonh e dne inoiie wkoogs y iend oae y o lns chseufnla(aht s'enjeu inophb  ouoe fok os ai i cee, e s roel''n aleniia oafthriannd hheoee 'ooohitgt ynenneens whto8olhmhootwsoudeuu ? y y
EPOCH =  22
iter:  0 	|	 ie greobh of ton b onxe z wo own t se iie i wse l t b b t t yacba we gwhml -oele s y bnweue

EPOCH =  39
iter:  0 	|	 id herq fo  tit,d woyo's'iugoudou se ol'dhahee iyitovewlnakeon t cathe bata, s taho (onsovt ts s se je at awousanse indas thmen the it im fe,yhe atop toi g be seis w s th s,hg[nthat tonte isfout l f t
iter:  100 	|	 ws inhe c o yoremeo y wsdhe f tine ite  yoifhe soco lh in ssayhprthe cis tng lleoin inon inhe aual'ng i''&shey, r itdifnhe we ctoloukilor y f in n'si, ts wuujobon phe tant s rli me heccigfirh ld getye
EPOCH =  40
iter:  0 	|	 n og'noinaee'it, ti doiastau re f y and bhe wig wagigw ito whr t m seruonolbhnyp arakayoe, at zhe llou t sine wassnd w's t y caan t torj'san tyrt'ffnt inon asdrfarb iy'tibuy k ou w tondo inuhe wliglae
iter:  100 	|	 r r sun ale'y wathe s w u younerot anw f lye  paonot i dmtrsijtin yoice y'tout titonou&eerat w saanthawi tho[ok tac weghe stouo yo wgheghe i, hofsano t, t s teo wap in'l:ohe ir t l t d you rthidves ce
EPOCH =  41
iter:  0 	|	 y l m o d t inn it s t wanin t ly r ib dhe s ao'nathee t y la y ar t l t s iillo w se ino t

EPOCH =  58
iter:  0 	|	 kusse  what ithinveesri'tase y n f'wrshi s pous ttat ,'tdakar d yoime younut d angrt ou lf tolorofri d fvat it s sour s  ou me lool'melsalc indule t inu'cere ithaydhouj y tilir omert isitnon'the inghe
iter:  100 	|	 g amabe ivet pe ds rethe p lit youfrghe frad s ui'wan'de lhe hnd se ychesallfet ingowan dikop ad t vacarm ve obge y oin came d t bi, ol i'o , i't urt i ithaweofunsino ilm ove souwyo a, s, be w yove t 
EPOCH =  59
iter:  0 	|	 d mu lud ak t hedo tnu u s licathl i l bo wf d thi a sheu f lat  tinn  &sgof bd g the d at roly meliyoldoud caw t doovee ou c ny werre sonng yougy re thens se"tat tho yove owotb youbet t bass ing so l
iter:  100 	|	 'thee''thacalof bat cenghe ma oyovean  ins ol be was le  waninisid (ali t ug y t  y whisd 'bu me dou in l  ctghe m onke wes trthe ikove cee lonn, tolt r f idesx inr d i  iy yondor y cas2taktong on yo 
EPOCH =  60
iter:  0 	|	 yov atbbua  y ou't m)ououtove lojor bstous the was he, int mis  ynlirce fg't ranyousouou ce

EPOCH =  77
iter:  0 	|	 meme s wdito my wannoucsac i't thelyoneef m alfi'me yvof llst tke keind dicang, d me lthe metan i alou'the bous me iyove iss louitritove he nnne he songghousougafhtst i d t in'the bo ct thtse tht wan 
iter:  100 	|	 'ther ik mt, watamthehe ighy p ayo ce re  ini'fi al p or yi'tinlinou wange abe, ne ininy ie lyit  wo jiyou povele se r the s lindtokeayot yokie h, igl8it inghd in u o ouryonoll ir mlheie yose'an sesha
EPOCH =  78
iter:  0 	|	 athe taof y tht se igithe iyofbacanwt, t y alog i y alli t thm fay tofin me  c mat chean'start thede hjlin't onsauseca, the llat thinghisesy s, led t takl iveicht h pthwayone you t tctay t r t'gonoowa
iter:  100 	|	 e yome inda au i se what kyon've neeiy ay b atshelin kecig, t s p los owht me is i de wht prar me a  nninod d sidd ohicase cyo s a ke t'pve yoloumeng acather r is yousa awhe min'yon'tot dous s rive do
EPOCH =  79
iter:  0 	|	  t's b le f lli't tota'me a anti theyof trhinghofr, w, t i''nstoume i'se none whitsanikecon

...

iter:  100 	|	 are inee thyot miscankerofe l i'zy, me  t'che'tive sents ighin'caig shele  ce fo ingoliffowa zole anatonend wonofo  t  sh t nazor  pake tsous  is m s writ n a tha pari i'lomy ithike e wadof  k men m w
EPOCH =  583
iter:  0 	|	 re bul le i lit obat en t y t ou't'rive aket t our ake, i, wr tot wan mitserouththe packee t blor woull i't'veyootche opaknoure wanou, it inga lonndout t terd w d tin lit's hi t ithe nesa athtabous do
iter:  100 	|	 'tofie take ri'ca whay yowowonn u ayo l wachorindowa ton't ie kshousacare ca a myon'met tinghanos s y y m  ikng i'cerdout cau ing whadive t r ind aloke imigindou i, d a y 'stest gincat itht ke bour i 
EPOCH =  584
iter:  0 	|	 sodes that m oout ju wayoffu, soun atathe aneanou isrithi'r zon s ke ooffelem t ldog bldous tiseht watovedounghe d th, inn ton ht, is ayo ff, y yovig wak i'vebahout t se ine y dat be ind wand t beghin
iter:  100 	|	 ou y cet'rau  pletheikee i't nd it, allinadouse ake y wa le lout t llinknd hes yonaree meeti t tath

iter:  0 	|	 t au you, tch it m ke:strokst findighlike ayoi'tat booopll it ithe lyout t finantowhe pre u then aitoofak t t it s all to of, s le yow tan'r s uri tame nkew zouplay w  wets caunen od nd be way i ay yo
iter:  100 	|	 gs  r inake itoware l a you s lit me thitidile, at ngheanedo y, w fof zerereritcake wah ysesisher ge tht idi ou in'stoust s, wh ay yoor brakee my, lith gis t ig ate tyingaverayer t  t prdy ay ke  s sh
EPOCH =  602
iter:  0 	|	 ffu t'tat baise of wad 'vedofff t p th, oun take myowal in'ceayouth song light wanghigo me l t thimolis  annive tou waththeawo w tak re sbus t byou'tr odere ins londake  y cl iveshe t sor y a anto oor
iter:  100 	|	 iol i whe  taig of is at'the ino t is alenghit'donn buco ge inght  fir ng t llf lo she yold y alfar po lsour thrt rinnehayo t  ba w nd yout'cau, youcin'ru igi yo you d [therong m wa are id walod p y k
EPOCH =  603
iter:  0 	|	 l y it f toff t cayonk natatht't be t ile t t ove cayonghers wale ou wat bo toff ket ig t bothl w l  

EPOCH =  620
iter:  0 	|	 acane tand abe t he, t t aw i bakit  't wake igor ts beng l ttalsa it yof t?on fo owat wa:nove t i it se fffo y, a thee tr whiso r y y t y t walowallle gat'thea tayo gof tis in's  lin yoousas, t i oth
iter:  100 	|	 dyefer abous sson t is reveef yofodyean' t i he ne igharof ame ivea sithr wamyon sis'r plll yout t n  i'chey ie zore  lin l t ayout m tiker out  a s mee or nghithtandr ght amyout s ondou'callon thi, b
EPOCH =  621
iter:  0 	|	 m he cat igodimy igit itomof t t t beealo ow ilwese inof me myor f ti t t i on'boven s wird nyofusononisire lo s you tof, itots ntcay ke s f ind itone anyof whan'ayofaceton toushes w whtindo'shme thto
iter:  100 	|	 au'st orndonngso ts wa dondinde inghangs, tore ithighi hig l teyor m a mou l t t ane watowars ind bu the aro t t feralowand cl g a tore t l lishe ig, s, fishe nonoke iouro riontongowor caburike odar w
EPOCH =  622
iter:  0 	|	 k isal  meat nnt m t the d prigindopi dit bahted ang lo, it t'ronde (y, s ingan'teame on

iter:  100 	|	 'dpa ayon p  ive sthe in'ss are d whall s ta ke ld ikelar ine she fineamo ] fulloke aknabex inghighalour i ne it ightove s way, ee mel whthe? ighig kis ith au thorous s brir ainedithe is. athe d alan'
EPOCH =  639
iter:  0 	|	 k at me in ourome ig wa ala itin yoou'call galon los w offesitcaknllen t iok owal t t fe ig d opouth fote t y 've nd t f thaki kit me windout inngee  thoveau ba it m m ay d se f  che't witatha w wave 
iter:  100 	|	 gs bol of ndours oveurat  wouce onng bee me e in ale, gat  owal ll rd th, ike war bais whtht yone halealou we when anthayou liston t inn p g y yeri t zouthathe ous athi' y igoout, ffe bor he loshaby n
EPOCH =  640
iter:  0 	|	 hton yoneu athe loupazlowonnotrext in'then'rana ad aiveayomyo n'thithit w  thel mers t f 'me lot't, odirsen igh, ake t, n bomshlofo lleve it shton cmnt nyovithofff aseshe atobe " p asof bout.oreavee w


KeyboardInterrupt: 